# PASSO 00 IMPORTS

In [65]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error
import boto3
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


# PASSO 01 DATA DESCRIPTION

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Carregar os dados (supondo que já tenha sido coletado)
df = pd.read_parquet('../data/raw/historical_data.parquet')

In [40]:
from pyathena import connect
import pandas as pd

def query_athena(query, database, region='us-east-1', output_location='s3://my-firehose-bucket/query_results/'):
    """
    Executa uma query no AWS Athena e retorna os resultados em um DataFrame do Pandas.

    :param query: A consulta SQL a ser executada no Athena.
    :param database: O nome do banco de dados no Athena.
    :param region: A região AWS onde o Athena está configurado.
    :param output_location: O local no S3 para onde os resultados da query serão salvos.
    :return: DataFrame com os resultados da consulta.
    """
    # Conexão com Athena
    conn = connect(s3_staging_dir=output_location, region_name=region, database=database)
    
    # Executando a query e retornando o resultado como um DataFrame
    df = pd.read_sql(query, conn)
    
    return df


In [42]:
query = 'SELECT * FROM "btc_data"."btc_last_year"'
database = 'btc_data'
  # Defina seu bucket S3 para os resultados da query

df = query_athena(query, database)

# Exibindo o resultado
print(df)


/tmp/ipykernel_373847/3728450170.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


                  timestamp    price (usd)
0   2024-11-17 22:12:00.543   89066.663848
1   2024-11-17 23:11:49.126   89574.747275
2   2024-11-18 00:07:35.977   89794.981793
3   2024-11-18 01:01:06.793   89903.467214
4   2024-11-18 02:03:56.913   90801.317670
..                      ...            ...
715 2024-12-17 18:00:09.610  107473.874029
716 2024-12-17 19:03:53.037  106620.368592
717 2024-12-17 20:02:40.683  106495.852000
718 2024-12-17 21:00:58.374  106867.016362
719 2024-12-17 21:19:56.000  106672.038239

[720 rows x 2 columns]


# FEATURE ENGINEERING

In [119]:
df['day_of_week'] = df['timestamp'].dt.dayofweek


# Extrair o número do dia no mês
df['day_of_month'] = df['timestamp'].dt.day

/tmp/ipykernel_373847/1191623834.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['day_of_week'] = df['timestamp'].dt.dayofweek
/tmp/ipykernel_373847/1191623834.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['day_of_month'] = df['timestamp'].dt.day


In [120]:
# Média móvel de 7 dias (curto prazo)
df['moving_avg_7'] = df['price (usd)'].rolling(window=7).mean()

# Média móvel de 30 dias (médio prazo)
df['moving_avg_30'] = df['price (usd)'].rolling(window=30).mean()

# Média móvel de 90 dias (longo prazo)
df['moving_avg_90'] = df['price (usd)'].rolling(window=90).mean()

/tmp/ipykernel_373847/3424091257.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['moving_avg_7'] = df['price (usd)'].rolling(window=7).mean()
/tmp/ipykernel_373847/3424091257.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['moving_avg_30'] = df['price (usd)'].rolling(window=30).mean()
/tmp/ipykernel_373847/3424091257.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [121]:
# Calcular a variação percentual diária (retorno)
df['daily_return'] = df['price (usd)'].pct_change()

# Calcular a variação absoluta diária
df['daily_return_abs'] = df['price (usd)'].diff()

/tmp/ipykernel_373847/91719867.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['daily_return'] = df['price (usd)'].pct_change()
/tmp/ipykernel_373847/91719867.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['daily_return_abs'] = df['price (usd)'].diff()


In [122]:
df['volatility'] = df['daily_return'].rolling(window=30).std()

/tmp/ipykernel_373847/4287875491.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['volatility'] = df['daily_return'].rolling(window=30).std()


In [123]:
# Função para calcular o RSI
def calculate_rsi(data, window):
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

# Calcular o RSI de 14 dias
df['RSI_14'] = calculate_rsi(df['price (usd)'], window=14)

/tmp/ipykernel_373847/3570803778.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['RSI_14'] = calculate_rsi(df['price (usd)'], window=14)


In [124]:
# Média exponencial de 7 dias
df['ema_7'] = df['price (usd)'].ewm(span=7, adjust=False).mean()

# Média exponencial de 30 dias
df['ema_30'] = df['price (usd)'].ewm(span=30, adjust=False).mean()

/tmp/ipykernel_373847/1185787216.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ema_7'] = df['price (usd)'].ewm(span=7, adjust=False).mean()
/tmp/ipykernel_373847/1185787216.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ema_30'] = df['price (usd)'].ewm(span=30, adjust=False).mean()


# Variable Filtering

In [125]:
df = df.dropna()

In [137]:
# Lista das variáveis derivadas
derived_columns = [
    'day_of_week',  'day_of_month', 
    'moving_avg_7', 'moving_avg_30', 'moving_avg_90'
]

# Filtrando apenas as variáveis derivadas
df_derived = df[derived_columns]

# Exibindo as primeiras linhas das variáveis derivadas
print(df_derived.head())

     day_of_week  day_of_month  moving_avg_7  moving_avg_30  moving_avg_90
178            0            25  98062.335735   97616.479249   98184.505194
179            0            25  98209.079158   97618.809577   98203.762841
180            0            25  98288.869779   97626.316327   98222.998790
181            0            25  98330.577655   97623.906333   98234.567381
182            0            25  98216.174337   97597.870450   98229.295823


In [138]:
scaler = StandardScaler()
# Separar variáveis independentes (apenas as derivadas)
X_derived = df_derived

# Variável dependente (preço)
y = df['price (usd)']

# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_derived, y, test_size=0.3, shuffle=False)

# Normalizar as variáveis independentes
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [139]:
# Inicializar e treinar o modelo de regressão linear
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

# Prever no conjunto de teste
y_pred = linear_model.predict(X_test)

# Avaliar o modelo
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error (Random Forest): {mse}')
print(f'Root Mean Squared Error (RMSE) (Random Forest): {mse}')
print(f'Mean Absolute Error (MAE) (Random Forest): {mae}')
print(f'R² (Random Forest): {r2}')

Mean Squared Error (Random Forest): 768278.257355873
Root Mean Squared Error (RMSE) (Random Forest): 768278.257355873
Mean Absolute Error (MAE) (Random Forest): 663.6880838194778
R² (Random Forest): 0.8887768247628055


In [150]:
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor

# ------------------ Treinamento com XGBoost ------------------
xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.4, random_state=42)
xgb_model.fit(X_train, y_train)

# Previsão com XGBoost
xgb_y_pred = xgb_model.predict(X_test)

# Avaliação do XGBoost
xgb_mse = mean_squared_error(y_test, xgb_y_pred)
xgb_rmse = np.sqrt(xgb_mse)
xgb_mae = mean_absolute_error(y_test, xgb_y_pred)
xgb_r2 = r2_score(y_test, xgb_y_pred)

print("------ XGBoost Results ------")
print(f'Mean Squared Error (XGBoost): {xgb_mse}')
print(f'Root Mean Squared Error (RMSE) (XGBoost): {xgb_rmse}')
print(f'Mean Absolute Error (MAE) (XGBoost): {xgb_mae}')
print(f'R² (XGBoost): {xgb_r2}')


# ------------------ Treinamento com Gradient Boosting ------------------
gb_model = GradientBoostingRegressor(n_estimators=10, learning_rate=0.01, random_state=42)
gb_model.fit(X_train, y_train)

# Previsão com Gradient Boosting
gb_y_pred = gb_model.predict(X_test)

# Avaliação do Gradient Boosting
gb_mse = mean_squared_error(y_test, gb_y_pred)
gb_rmse = np.sqrt(gb_mse)
gb_mae = mean_absolute_error(y_test, gb_y_pred)
gb_r2 = r2_score(y_test, gb_y_pred)

print("\n------ Gradient Boosting Results ------")
print(f'Mean Squared Error (Gradient Boosting): {gb_mse}')
print(f'Root Mean Squared Error (RMSE) (Gradient Boosting): {gb_rmse}')
print(f'Mean Absolute Error (MAE) (Gradient Boosting): {gb_mae}')
print(f'R² (Gradient Boosting): {gb_r2}')

------ XGBoost Results ------
Mean Squared Error (XGBoost): 24375259.972185727
Root Mean Squared Error (RMSE) (XGBoost): 4937.130742869357
Mean Absolute Error (MAE) (XGBoost): 4086.4106113158796
R² (XGBoost): -2.5287915353340327

------ Gradient Boosting Results ------
Mean Squared Error (Gradient Boosting): 32738091.838436842
Root Mean Squared Error (RMSE) (Gradient Boosting): 5721.721055629752
Mean Absolute Error (MAE) (Gradient Boosting): 5096.995023140828
R² (Gradient Boosting): -3.7394736094831043


In [142]:
import joblib
import os

# Caminho local para salvar o modelo
local_model_path = '../models/rl_model.pkl'


# Salvar o modelo
joblib.dump(linear_model, local_model_path)

print(f"Modelo salvo localmente em: {local_model_path}")


Modelo salvo localmente em: ../models/rl_model.pkl


In [143]:
import boto3

class S3Services:
    def __init__(self, bucket_name):
        self.s3_client = boto3.client('s3')
        self.bucket_name = bucket_name

    def upload_file(self, file_name, s3_key):
        """
        Faz upload de um arquivo para um bucket S3.
        :param file_name: Caminho local do arquivo.
        :param s3_key: Caminho no S3 onde o arquivo será salvo.
        """
        try:
            self.s3_client.upload_file(file_name, self.bucket_name, s3_key)
            print(f"Arquivo {file_name} enviado com sucesso para {s3_key}.")
        except Exception as e:
            print(f"Erro ao enviar o arquivo para o S3: {str(e)}")

In [144]:
# Inicializar o serviço S3
bucket_name = 'my-firehose-bucket'  # Substitua pelo nome do seu bucket S3
s3_service = S3Services(bucket_name)

# Caminho no S3 onde o arquivo será salvo
s3_model_path = 'ml_models/rl_model.pkl'

# Upload do modelo para o S3
s3_service.upload_file(local_model_path, s3_model_path)

Arquivo ../models/rl_model.pkl enviado com sucesso para ml_models/rl_model.pkl.


In [145]:
X_train.columns

Index(['day_of_week', 'day_of_month', 'moving_avg_7', 'moving_avg_30',
       'moving_avg_90'],
      dtype='object')